In [1]:
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Average, Dropout
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.datasets import fashion_mnist

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

/home/fineral/.conda/envs/myenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
x_train = x_train / 255.
x_test = x_test / 255.
encoded_y_train = to_categorical(y_train, num_classes=10)

In [4]:
shape = 'x_train shape: {} | encoded_y_train shape: {}\n'
shape += 'x_test shape : {} | y_test shape : {}'
print(shape.format(x_train.shape, encoded_y_train.shape, x_test.shape, y_test.shape))

x_train shape: (60000, 28, 28) | encoded_y_train shape: (60000, 10)
x_test shape : (10000, 28, 28) | y_test shape : (10000,)


In [5]:
def compile_and_train(model, num_epochs): 
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train, y=encoded_y_train, batch_size=32, epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    return history

In [6]:
x_train = x_train.reshape(60000,28,28,1)
input_shape = x_train[0,:,:,:].shape # 28 by 28
print(x_train.shape)
model_input = Input(shape=input_shape)

(60000, 28, 28, 1)


In [7]:
def third_cnn(model_input):
    x = Conv2D(96, kernel_size=(5, 5), activation='sigmoid', padding='same')(model_input)
    x = MaxPooling2D(pool_size=(5, 5), strides = 2)(x)
    x = Conv2D(192, (5, 5), activation='sigmoid', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(5, 5), strides = 2)(x)
    x = Conv2D(192, (5, 5), activation='sigmoid', padding = 'same')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='third_cnn')
    
    return model

In [8]:
third_model = third_cnn(model_input)

In [12]:
_ = compile_and_train(third_model, num_epochs=8)

Train on 48000 samples, validate on 12000 samples
Epoch 1/8
48000/48000 [==============================] - 13s 263us/step - loss: 0.3351 - acc: 0.8748 - val_loss: 0.3344 - val_acc: 0.8750
Epoch 2/8
48000/48000 [==============================] - 13s 263us/step - loss: 0.3252 - acc: 0.8777 - val_loss: 0.3468 - val_acc: 0.8708
Epoch 3/8
48000/48000 [==============================] - 13s 263us/step - loss: 0.3182 - acc: 0.8807 - val_loss: 0.3337 - val_acc: 0.8739
Epoch 4/8
48000/48000 [==============================] - 12s 259us/step - loss: 0.3046 - acc: 0.8861 - val_loss: 0.3225 - val_acc: 0.8770
Epoch 5/8
48000/48000 [==============================] - 13s 262us/step - loss: 0.3007 - acc: 0.8872 - val_loss: 0.3145 - val_acc: 0.8832
Epoch 6/8
48000/48000 [==============================] - 12s 259us/step - loss: 0.2923 - acc: 0.8904 - val_loss: 0.3263 - val_acc: 0.8832
Epoch 7/8
48000/48000 [==============================] - 13s 262us/step - loss: 0.2859 - acc: 0.8924 - val_loss: 0.2887 - 